In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('string-network-1.csv')
df.head()

In [ ]:
# get list of nodes

nodes = df.node1.unique()

In [ ]:
nodes

In [ ]:
# build dict of node inputs

node_inputs = {}
for node in nodes:
    inputs = list(df[df.node1 == node].node2)
    node_inputs[node] = inputs

In [ ]:
# construct boolean rules as list
rules = []
for key, value in node_inputs.items():
    rule = key + ' *= '
    rule = rule + ' and '.join(value)
    rules.append(rule)

In [ ]:
# construct initial conditions
initial_conditions = []
for node in nodes:
    initial_condition = node + ' = Random'
    initial_conditions.append(initial_condition)

In [ ]:
# export model
definition = '#initial conditions\n'+'\n'.join(initial_conditions)+'\n\n#rules\n'+'\n'.join(rules)
fp = open('model.txt', 'w')
fp.write(definition)
fp.close()

In [ ]:
inp = file('model.txt').read()

In [ ]:
#  print(inp)

In [ ]:
def string2definition(tabular_text_output, initial_value):
    """
    model 1
    """
    df = pd.read_csv(tabular_text_output)
    
    # get list of nodes
    nodes = df.node1.unique()
    
    # build dict of node inputs
    node_inputs = {}
    for node in nodes:
        inputs = list(df[df.node1 == node].node2)
        node_inputs[node] = inputs
        
    # construct boolean rules as list
    rules = []
    for key, value in node_inputs.items():
        rule = key + ' *= '
        rule = rule + ' or '.join(value)
        rules.append(rule)
        
    # construct initial conditions
    initial_conditions = []
    initial_value = 'True' if initial_value else 'False'
    for node in nodes:
        initial_condition = node + ' = ' + initial_value
        initial_conditions.append(initial_condition)
        
    # definition
    return (
        '#initial conditions\n'+
        '\n'.join(initial_conditions)+         
        '\n\n'+
        '#rules\n'+
        '\n'.join(rules)
    )        

In [ ]:
def add_process_edgelist(definition, edgelist, initial_value):
    """
    model 1
    """
    df = pd.read_csv(edgelist)
    
    # get list of nodes
    nodes = df.process.unique()

    # build dict of node inputs
    node_inputs = {}
    for node in nodes:
        inputs = list(df[df.process == node].node)
        node_inputs[node] = inputs
    
    # remove input nodes not in the network (not modelled)
    for inputs in node_inputs.values():
        for input in inputs:
            if input not in definition: inputs.remove(input)

    # construct boolean rules as list
    rules = []
    for key, value in node_inputs.items():
        rule = key + ' *= '
        rule = rule + ' and '.join(value)
        rules.append(rule)
        
    # construct initial conditions
    initial_conditions = []
    initial_value = 'True' if initial_value else 'False'
    for node in nodes:
        initial_condition = node + ' = ' + initial_value
        initial_conditions.append(initial_condition)
        
    # definition
    return (
        definition + 
        '\n\n'+
        '#process node initial conditions\n'+
        '\n'.join(initial_conditions)+
        '\n\n'+
        '#process node rules\n'+
        '\n'.join(rules)
    )

In [ ]:
def add_mtb_edgelist(definition, mtb_edgelist, initial_value):
    """
    model 1
    """
    df = pd.read_csv(mtb_edgelist)

    # get list of nodes
    target_nodes = df.node.unique()
    
    # build dict of node inputs
    node_inputs = {}
    for node in target_nodes:
        inputs = list(df[df.node == node].mtb)
        node_inputs[node] = inputs
    
    # remove factors without target nodes in the network (not modelled)
    mtb_nodes = []
    for node, mtb in node_inputs.items():
        if node not in definition:
            del node_inputs[node]
        else:
            mtb_nodes+=mtb
            mtb_nodes = list(set(mtb_nodes)) #unique values

    # construct boolean rules as list
    rules = []
    for target_node, mtb in node_inputs.items():
        rule = target_node + ' *= ' + target_node # add the inhibition rule recursively
        rule = rule + ' and not (' +' or '.join(mtb) + ')'
        rules.append(rule)

    # construct initial conditions
    initial_conditions = []
    initial_value = 'True' if initial_value else 'False'
    for node in mtb_nodes:
        initial_condition = node + ' = ' + initial_value
        initial_conditions.append(initial_condition)
        
    # definition
    return (
        definition + 
        '\n\n'+
        '#mtb node initial conditions\n'+
        '\n'.join(initial_conditions)+
        '\n\n'+
        '#mtb update rules\n'+
        '\n'.join(rules)
    )

In [ ]:
definition = string2definition('string-network-1.csv', True)
definition = add_process_edgelist(definition,'string-1-process-edgelist.csv', True)
print add_mtb_edgelist(definition, 'mtb-edgelist.csv', False)

In [ ]:
# read in complexes and create a dict
# build dict of node inputs

df_complexes = pd.read_csv('string-1-complexes.csv')
complexes = df_complexes.complex.unique()
complexes

In [ ]:
complex_components = {}
for complex in complexes:
    inputs = list(df_complexes[df_complexes.complex == complex].node)
    complex_components[complex] = inputs
complex_components

In [ ]:
node_inputs

def reduce_complexes(node_inputs, complex_components):
    # remove nodes in a complex
    for components in complex_components.values():
        for node in node_inputs.keys():
            if node in components:
                del node_inputs[node]
                continue

    # replace complex nodes
    for node, inputs in node_inputs.items():
        for complex, components in complex_components.items():
            for i, input in enumerate(inputs):
                if input in components:
                    node_inputs[node][i] = complex
                    continue
        node_inputs[node] = list(set(node_inputs[node]))
    return node_inputs

node_inputs # get OR'd

In [ ]:
complex_components # get AND'd

In [ ]:
# check complex to complex

complex_inputs = {}
for complex, components in complex_components.items():
    inputs = []
    for component in components:
        inputs+=node_inputs[component]
    complex_inputs[complex] = inputs
complex_inputs=reduce_complexes(complex_inputs, complex_components)
for complex, inputs in complex_inputs.items():
    inputs.remove(complex)
complex_inputs

In [ ]:
def get_complex_inputs(node_inputs, complex_components):
    complex_inputs = {}
    for complex, components in complex_components.items():
        inputs = []
        for component in components:
            inputs+=node_inputs[component]
        complex_inputs[complex] = list(set(inputs))
    complex_inputs=reduce_complexes(complex_inputs, complex_components)
    for complex, inputs in complex_inputs.items():
        inputs.remove(complex)
    return complex_inputs

In [ ]:
    complex_inputs = get_complex_inputs(node_inputs, complex_components)

complex_inputs

In [ ]:
# write export helper functions

def string2definition2(tabular_text_output, complexes, initial_value):
    df = pd.read_csv(tabular_text_output)
    df_complexes = pd.read_csv(complexes)

    # get list of nodes 
    nodes = list(df.node1.unique())
    
    # build dict of node inputs
    node_inputs = {}
    for node in nodes:
        inputs = list(df[df.node1 == node].node2)
        node_inputs[node] = inputs
        
    # get list of complexes 
    complexes = list(df_complexes.complex.unique())

    # build dict of components
    complex_components = {}
    for complex in complexes:
        inputs = list(df_complexes[df_complexes.complex == complex].node)
        complex_components[complex] = inputs

    # add complex inputs
    complex_inputs = get_complex_inputs(node_inputs, complex_components)
    
    # reduce complexes
    node_inputs = reduce_complexes(node_inputs, complex_components)
    
    # OR node inputs
    rules = []
    for node, inputs in node_inputs.items():
        rule = node + ' *= '
        rule = rule + ' or '.join(inputs)
        rules.append(rule)
        
    # AND complex components
    for complex, components in complex_components.items():
        rule = complex + ' *= '
        rule = rule + ' and '.join(components)
        rules.append(rule)
    
    # OR complex inputs
    for complex, inputs in complex_inputs.items():
        rule = complex + ' *= '
        rule = rule + complex + ' or '+' or '.join(inputs)
        rules.append(rule)
        
    # initial conditions
    initial_value = 'True' if initial_value else 'False'
    initial_conditions = []
    for node in (nodes+complexes):
        initial_condition = node + ' = ' + initial_value
        initial_conditions.append(initial_condition)
        
    # definition
    definition = '#initial conditions\n'+'\n'.join(initial_conditions)+'\n\n#rules\n'+'\n'.join(rules)
    
    return definition
        

In [ ]:
definition2 = string2definition2('string-network-1.csv', 'string-1-complexes.csv', True)

In [ ]:
print definition2

In [ ]:
def add_process_edgelist2(definition, edgelist, complexes, initial_value):
    df = pd.read_csv(edgelist)
    df_complexes = pd.read_csv(complexes)

    # get list of nodes
    nodes = df.process.unique()

    # build dict of node inputs
    node_inputs = {}
    for node in nodes:
        inputs = list(df[df.process == node].node)
        node_inputs[node] = inputs
    
    # get list of complexes 
    complexes = list(df_complexes.complex.unique())

    # build dict of components
    complex_components = {}
    for complex in complexes:
        inputs = list(df_complexes[df_complexes.complex == complex].node)
        complex_components[complex] = inputs
    
    
    
    # reduce complexes
    node_inputs = reduce_complexes(node_inputs, complex_components)
    
    # construct boolean rules as list
    rules = []
    for key, value in node_inputs.items():
        rule = key + ' *= '
        rule = rule + ' and '.join(value)
        rules.append(rule)
        
    # initial conditions
    initial_value = 'True' if initial_value else 'False'
    initial_conditions = []
    for node in (nodes):
        initial_condition = node + ' = ' + initial_value
        initial_conditions.append(initial_condition)
        
    # definition
    return definition + '\n\n#process node initial conditions\n'+'\n'.join(initial_conditions)+'\n\n#process node rules\n'+'\n'.join(rules)


In [ ]:
print add_process_edgelist2(definition2, 'string-1-process-edgelist.csv', 'string-1-complexes.csv', True)